# Making the word embedding

Running the cells below reads in a text file called ``clean_data.txt`` of the form

    agent_name_speaker1 cleaned_quote1 
    agent_name_speaker2 cleaned_quote2 
    ...
    
and trains a word embedding from the text, using the gensim package. 

The word embedding is stored as ``word_embedding.emb`` using gensim's native model storing function, and can be loaded back into a gensim embedding via ``gensim.models.KeyedVectors.load(word_embedding.emb)``.

In [1]:
import numpy as np
from gensim.models import Word2Vec
from random import seed, shuffle
from gensim.models.callbacks import CallbackAny2Vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# By changing these seeds, randomness in the loading and training process
# will produce a different embedding each time.

data_seed = 42 
model_seed = 42

## 1. Define some components used for training

In [3]:
# This class is needed to print out the loss during training, which can be useful to monitor
# the convergence behaviour: if the loss decreases significantly between two epochs, the 
# model may not be fully trained yet.

class PrintLoss(CallbackAny2Vec):
    """Callback to print loss after each epoch."""

    def __init__(self):
        self.epoch = 1
        self.loss_to_be_subed = 0
        self.log = ""

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.log += 'Loss after epoch {}: {} \n'.format(self.epoch, loss_now)
        self.epoch += 1

In [4]:
# This class is a generator object. Each time it is called it returns a new line (i.e. data sample)
# from the file stored under "path_to_data". Most of the logic is a trick to perform pseudo-randomisation for large 
# input files.

class DataGenerator(object):
    def __init__(self, path_to_data,
                 share_data=1.,
                 chunk_size=10000,
                 random_buffer_size=100000,
                 data_seed=data_seed):
        
        """Iterator that loads lines from a (possibly large) file in a mildly randomised fashion. 
        
        A buffer stores a set of lines from the text file. The buffer is shuffled, and the first chunk 
        of lines is returned (that is, one such line is yielded each time the generator is called). The buffer 
        is filled up again with fresh lines and shuffled. This continues until no lines are left to fill the 
        buffer with, at which point the remaining lines are returned.
        
        Args:
            path_to_data (str): Full path to a data file with one preprocessed sentence/document per line.
            share_data (float): How much data to subsample - useful for bootstrapping.
            chunk_size (int): Return so many lines from the random buffer at once before filling it up again. Larger
                chunk sizes speed up training, but decrease randomness.
            random_buffer_size (int): Keep so many lines from the data file in a buffer which is shuffled before
                returning the samples in a chunk. Higher values take more RAM but lead to more randomness
                when sampling the data. A value equal to the number of all samples would lead to perfectly
                random samples.
            data_seed (int): Random seed for data loading.
        """
        if chunk_size > random_buffer_size:
            raise ValueError("Chunk size cannot be larger than the buffer size.")

        self.path_to_data = path_to_data
        self.share_of_original_data = share_data
        self.chunk_size = chunk_size
        self.random_buffer_size = random_buffer_size
        seed(data_seed)

    def __iter__(self):

        # load initial buffer
        buffer = []
        with open(self.path_to_data, "r") as f:

            reached_end = False

            # fill buffer for the first time
            for i in range(self.random_buffer_size):
                line = f.readline().strip().split(" ")
                if not line:
                    reached_end = True
                    break
                buffer.append(line)

            while not reached_end:

                # randomise the buffer
                shuffle(buffer)

                # remove and return chunk from buffer
                for i in range(self.chunk_size):
                    # separate non-bootstrap case here for speed
                    if self.share_of_original_data == 1.0:
                        yield buffer.pop(0)
                    else:
                        # randomly decide whether this line is in
                        # the bootstrapped data
                        if np.random.rand() > self.share_of_original_data:
                            buffer.pop(0)
                            continue
                        else:
                            yield buffer.pop(0)

                # fill up the buffer with a fresh chunk
                for i in range(self.chunk_size):
                    line = f.readline()
                    if not line:
                        reached_end = True
                        break
                    else:
                        buffer.append(line.strip().split(" "))

            # if end of file has been reached
            # yield all elements left in the buffer
            # in random order
            shuffle(buffer)
            for el in buffer:
                yield el

## 2. Training the embedding

Training can now be done in a few lines. The hyperparameters we use were found by empirically optimising various embeddings.

In [5]:
training_generator = DataGenerator("clean_data.txt")

model = Word2Vec(
    sentences = training_generator, 
    vector_size = 250, # number of dimensions that the word vectors will have
    window = 10,  # maximum distance between the current and predicted word
    sg = 1,  # use skip-gram (semantic learning) as training algorithm
    workers = 8,  # number of threads for training the model
    min_count = 1,  # ignores all words with total frequency lower than this  # CHANGE BACK TO 5 
    sorted_vocab = 1, # sort the words in the resulting embedding
    seed = model_seed, # use a random seed for reproducability
    epochs = 30,  # number of times training goes through the data
    compute_loss = True, # print the loss in each epoch
    callbacks = [PrintLoss()])
    
# normalise the word vectors
model.wv.init_sims()

# extract and save embedding
emb = model.wv
emb.save("word_embedding.emb")

2022-07-05 16:56:08,192 : INFO : collecting all words and their counts
2022-07-05 16:56:08,795 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-07-05 16:56:09,173 : INFO : PROGRESS: at sentence #10000, processed 11977 words, keeping 1126 word types
2022-07-05 16:56:09,180 : INFO : PROGRESS: at sentence #20000, processed 23826 words, keeping 1848 word types
2022-07-05 16:56:09,188 : INFO : PROGRESS: at sentence #30000, processed 35872 words, keeping 2482 word types
2022-07-05 16:56:09,198 : INFO : PROGRESS: at sentence #40000, processed 47465 words, keeping 2895 word types
2022-07-05 16:56:09,207 : INFO : PROGRESS: at sentence #50000, processed 59645 words, keeping 3347 word types
2022-07-05 16:56:09,218 : INFO : PROGRESS: at sentence #60000, processed 71582 words, keeping 3632 word types
2022-07-05 16:56:09,230 : INFO : PROGRESS: at sentence #70000, processed 83310 words, keeping 3824 word types
2022-07-05 16:56:09,241 : INFO : PROGRESS: at sentence #8000

Loss after epoch 1: 165646.484375


2022-07-05 16:56:11,501 : INFO : EPOCH 1: training on 119096 raw words (23026 effective words) took 1.0s, 23284 effective words/s


Loss after epoch 2: 128048.828125


2022-07-05 16:56:12,503 : INFO : EPOCH 2: training on 119096 raw words (22946 effective words) took 1.0s, 23017 effective words/s


...

Loss after epoch 28: 79514.25


2022-07-05 16:56:41,260 : INFO : EPOCH 28 - PROGRESS: at 42.01% examples, 9554 words/s, in_qsize 1, out_qsize 0
2022-07-05 16:56:41,371 : INFO : EPOCH 28: training on 119096 raw words (23077 effective words) took 1.1s, 20618 effective words/s


Loss after epoch 29: 59417.75


2022-07-05 16:56:42,387 : INFO : EPOCH 29 - PROGRESS: at 25.18% examples, 5779 words/s, in_qsize 0, out_qsize 2
2022-07-05 16:56:42,483 : INFO : EPOCH 29: training on 119096 raw words (23000 effective words) took 1.1s, 20782 effective words/s
2022-07-05 16:56:42,484 : INFO : Word2Vec lifecycle event {'msg': 'training on 3572880 raw words (689867 effective words) took 33.0s, 20882 effective words/s', 'datetime': '2022-07-05T16:56:42.484709', 'gensim': '4.2.0', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-121-generic-x86_64-with-glibc2.29', 'event': 'train'}
2022-07-05 16:56:42,485 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=4143, vector_size=250, alpha=0.025>', 'datetime': '2022-07-05T16:56:42.485935', 'gensim': '4.2.0', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-121-generic-x86_64-with-glibc2.29', 'event': 'created'}
/tmp/ipykernel_34925/2139498310.py:17: DeprecationWarning:

Loss after epoch 30: 64538.75
